In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
model = models.resnet18()

## Print out the parameter names and shapes

In [3]:
for name, param in model.named_parameters():
  print(name, param.shape)

conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight torch.Size([128])
layer2.0.bn1.bias torch.Size([128])
layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight torch.Size([128])
layer2.0.bn2.bias torch.Size([128])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight tor

## To accelerate foward propagation and reduce memory, close the require_grads to do forward propagation

In [4]:
input_ = torch.rand(1,3,128,128)

In [5]:
for param in model.parameters():
  param.requires_grad = False

In [6]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## Build models in a more concise way

Input into a linear layer, than 10 layers of linear(10,10) with batch normalization

In [7]:
input_ = torch.randn(10,12)
Linear  = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [15]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.sequential = nn.Sequential(
        nn.Linear(12,10),
        nn.BatchNorm1d(10)
    )
    self.repeat_linear = nn.ModuleList([
        nn.Sequential(
            nn.Linear(10,10),
            nn.BatchNorm1d(10)
        ) for i in range(10)
    ])
    self.output = nn.Linear(10,3)
  def forward(self, x):
      x = self.sequential(x)
      for layer in self.repeat_linear:
        x = layer(x)
      x = self.output(x)
      return x

In [16]:
model = Model()

In [17]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_fe

In [18]:
input_ = torch.randn(10,12)
output = model(input_)

In [19]:
output

tensor([[ 0.5022, -0.4115, -0.8339],
        [-0.7806, -0.2634, -0.5223],
        [ 0.0512,  0.4503,  0.2174],
        [ 0.3761, -0.9821, -0.8430],
        [ 0.0935,  0.7243, -0.1161],
        [ 0.0458,  0.0934,  0.1658],
        [-0.7093, -0.4647, -0.4169],
        [ 0.1653, -0.9580,  0.1253],
        [ 0.5702,  0.1886, -0.0508],
        [ 1.2629, -1.4703,  0.0396]], grad_fn=<AddmmBackward>)